In [699]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymongo as pm
import pprint
from enum import Enum
from datetime import datetime, timedelta
import pytz

In [700]:

client = pm.MongoClient('bigdatadb.polito.it',                     
                        ssl=True,                     
                        authSource = 'carsharing',                     
                        username = 'ictts',                     
                        password ='Ict4SM22!',                     
                        tlsAllowInvalidCertificates=True) 
db = client['carsharing'] 

#Choose the DB to use 
permenant_booking = db['PermanentBookings']
permenant_parking = db['PermanentParkings']
enjoy_permenant_booking = db['enjoy_PermanentBookings']
enjoy_permenant_parking = db['enjoy_PermanentParkings']

#ENUM of cities
class CITY_ENUM(Enum):
    TO = 'Torino'
    SEA = 'Seattle'
    STU = 'Stuttgart'
class CITY_TIMEZONES(Enum):
    TO = 'Europe/Rome'
    SEA = 'America/Los_Angeles'
    STU = 'Europe/Berlin'

def get_start_end_unix_zone(timezone):
    start_timestamp = datetime(2018, 1, 1,0,0,0,0, pytz.timezone(timezone)).timestamp()
    end_timestamp  = datetime(2018, 1, 31,23,59,59,0, pytz.timezone(timezone)).timestamp()
    return start_timestamp,end_timestamp    

#date starts from 01/01/2018 to 31/01/2018 1514761200 - 1517353200
# start_unix_time = datetime.datetime.strptime("01/01/2018", "%d/%m/%Y").timestamp()
# end_unix_time = datetime.datetime.strptime("01/02/2018", "%d/%m/%Y").timestamp()

In [701]:
#pipeline for getting the data for the rentals with the filteration of the data
#too short and too long rentals are filtered out
#considered if car is moved
#grouped by date and hour
def filter_pipeline(city,start_unix_time,end_unix_time):
    return [
    {
        '$match': {
            'city': city,
            'init_time': {
                '$gte': start_unix_time,
                '$lt': end_unix_time
            },
            'final_time': {
                '$gte': start_unix_time,
                '$lt': end_unix_time
            }
        }
    },
    {
        '$project': {
            '_id': 0,
            'duration': {
                '$divide': [
                    { '$subtract': ['$final_time', '$init_time'] },
                    60  # Divide by 60 to convert seconds to minutes
                ]
            },
            'day': {'$dayOfMonth': '$init_date'},
            'hour': {'$hour': '$init_date'},
            'date': {
                '$dateToString': {
                    'format': '%Y-%m-%d',
                    'date': '$init_date'
                }
            },
            'moved': {
                '$ne':[
                    {"$arrayElemAt": [ "$origin_destination.coordinates", 0]},
                    {"$arrayElemAt": [ "$origin_destination.coordinates", 1]}
                 ]
            }
        }
    },
    {
        '$match': {
            'moved': True,
            'duration':{'$gt':5, '$lt':180},
                
        }
    },
    {
        '$group':{
            '_id': {'day': '$day', 'hour': '$hour', 'date': '$date'},
            'total_count': {'$sum': 1},
        }
    },
    {
        '$sort': {
            '_id': 1,
        }
    },
]


### Getting the data from Database

In [702]:
TO_Data = list(enjoy_permenant_booking.aggregate(filter_pipeline(CITY_ENUM.TO.value,
          get_start_end_unix_zone(CITY_TIMEZONES.TO.value)[0],get_start_end_unix_zone(CITY_TIMEZONES.TO.value)[1])))
SEA_Data = list(permenant_booking.aggregate(filter_pipeline(CITY_ENUM.SEA.value,
          get_start_end_unix_zone(CITY_TIMEZONES.SEA.value)[0],get_start_end_unix_zone(CITY_TIMEZONES.SEA.value)[1])))
STU_Data = list(permenant_booking.aggregate(filter_pipeline(CITY_ENUM.STU.value,
          get_start_end_unix_zone(CITY_TIMEZONES.STU.value)[0],get_start_end_unix_zone(CITY_TIMEZONES.STU.value)[1])))
cities_data_array = [(CITY_ENUM.TO.value,TO_Data),(CITY_ENUM.SEA.value,SEA_Data),(CITY_ENUM.STU.value,STU_Data)]

#### checking if there are missing records - lwngth must be 744

In [703]:
print("TO_Data",len(TO_Data))
print("SEA_Data",len(SEA_Data))
print("STU_Data",len(STU_Data))

TO_Data 744
SEA_Data 726
STU_Data 735


##### Dropping the _id col and flattening the data

In [704]:
def dfModifier(city_list):
  df = pd.DataFrame(city_list, columns =['_id', 'total_count'])
  df['date'] = df['_id'].apply(lambda x: x['date'])
  df['day'] = df['_id'].apply(lambda x: x['day'])
  df['hour'] = df['_id'].apply(lambda x: x['hour'])
  df['myIndex'] = (df['day']-1)*24 + (df['hour']+1)
  df.drop(['_id'], axis=1, inplace=True)
  return df
#day | hour
#1   | 0 -> day*24 + hour => 1*24 + 0 = 24
#1   | 1 -> day*24 + hour => 1*24 + 1 = 25
#1   | 2 -> day*24 + hour => 1*24 + 2 = 26
#day | hour
#0   | 1 -> day*24 + hour => 0*24 + 1 = 1
#0   | 2 -> day*24 + hour => 0*24 + 2 = 2
#0   | 3 -> day*24 + hour => 0*24 + 3 = 3

TO_df = dfModifier(TO_Data)
SEA_df = dfModifier(SEA_Data)
STU_df = dfModifier(STU_Data)
cities_df_array = [(CITY_ENUM.TO.value,TO_df),(CITY_ENUM.SEA.value,SEA_df),(CITY_ENUM.STU.value,STU_df)]


##### finding the mising values and filling them with Mean of Col

In [705]:
def fillMissingValues(df:pd.DataFrame):
  missingValues=set(np.arange(1,31*24+1)).difference(set(df['myIndex']))
  dfMean = round(np.mean(df['total_count']))
  print("Missing values are:", len(missingValues), missingValues)
  df2 = df
  for value in missingValues:
    dayOfValue = int((value-1)/24)+1
    hourOfValue = (value-1)%24
    new_row = pd.DataFrame({'total_count':dfMean,'date':f'2018-01-{dayOfValue:02d}',
                            'day':dayOfValue,'hour':hourOfValue,'myIndex':value}, index =[0])
    df2 = pd.concat([new_row,df2.loc[:]]).reset_index(drop = True)
  df2.sort_values(by=['myIndex'], inplace=True)
  return df2

To_FilledValues = fillMissingValues(TO_df)
SEA_FilledValues = fillMissingValues(SEA_df)
STU_FilledValues = fillMissingValues(STU_df)
#remember to remove this line - it has never been used !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! TODO
# SEA_FilledValues.at[744,'date'] = '2018-01-31'

Missing values are: 0 set()
Missing values are: 18 {736, 737, 738, 739, 740, 741, 742, 743, 726, 727, 728, 729, 730, 731, 732, 733, 734, 735}
Missing values are: 9 {736, 737, 738, 739, 740, 741, 742, 743, 744}


In [707]:
print("To_FilledValues",len(To_FilledValues))
print("SEA_FilledValues",len(SEA_FilledValues))
print("STU_FilledValues",len(STU_FilledValues), STU_FilledValues[743:745])

To_FilledValues 744
SEA_FilledValues 744
STU_FilledValues 744    total_count        date  day  hour  myIndex
0           76  2018-01-31   31    23      744


#### Plotting the data and Rolling mean and std to check if the time series is stationary

In [582]:
def plotter(plotTitle, df:pd.DataFrame):
    mean = df['total_count'].rolling(window=24*7).mean()
    std = df['total_count'].rolling(window=24*7).std()
    plt.figure(figsize=(14, 6))
    plt.plot(df['myIndex'], mean, label='Rolling Mean', color='red')
    plt.plot(df['myIndex'], std, label='Rolling Std', color='green')
    plt.plot()
    plt.plot(df['myIndex'], df['total_count'], label='Rental', color='blue')
    plt.xlabel('Date')
    plt.ylabel('Total Count')
    plt.legend()
    plt.grid(True)
    plt.title(f'Total Counts in Dates and Hours in - {plotTitle}')
    plt.grid(True)
    plt.savefig(f'{plotTitle}-Roolings-mean-std')
    plt.clf()

In [583]:
plotter('Torino',To_FilledValues)
plotter('Seattle',SEA_FilledValues)
plotter('Stuttgart',STU_FilledValues)

<Figure size 1400x600 with 0 Axes>

<Figure size 1400x600 with 0 Axes>

<Figure size 1400x600 with 0 Axes>

In [584]:
cities_df_array = [(CITY_ENUM.TO.value,To_FilledValues),(CITY_ENUM.SEA.value,SEA_FilledValues),(CITY_ENUM.STU.value,STU_FilledValues)]

,total_count,date,day,hour,myIndex
18,99.0,2018-01-01,1.0,0.0,1.0
19,66.0,2018-01-01,1.0,1.0,2.0
20,32.0,2018-01-01,1.0,2.0,3.0
21,13.0,2018-01-01,1.0,3.0,4.0
22,11.0,2018-01-01,1.0,4.0,5.0


## Computing the ACF and PACF

##### ACF Figure

In [630]:
from statsmodels.tsa.stattools import acf,pacf
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf

# Use ACF to find q.
# Use PACF to find p.

def ACF_PACF(city_data):
  # plot acf
  plt.figure(figsize=(6,4))
  plot_acf(city_data[1]["total_count"], lags=48)
  plt.title(f'Autocorrelation Function 48 Hours - {city_data[0]}')
  plt.xlabel('Lags')
  plt.grid(True)
  # plt.show()
  plt.savefig(f'{city_data[0]}-ACF')

  # plot pacf
  plt.figure(figsize=(6,4))
  plot_pacf(city_data[1]["total_count"], lags=48)
  plt.title(f'Partial Autocorrelation Function 48 Hours - {city_data[0]}')
  plt.xlabel('Lags')
  plt.grid(True)
  # plt.show()
  plt.savefig(f'{city_data[0]}-PACF')


In [664]:
for city_data in cities_df_array:
  ACF_PACF(city_data)

### ARIMA Model and Prediction

In [686]:
from statsmodels.tsa.arima.model import ARIMA

q = 4
p = 2
d = 0
train_samples = 24 * 7 * 3 # 3 weeks
test_size = 24 * 10 # 1 week
def Predictor(city_data):
  originalData = list(city_data[1]['total_count'][:train_samples])
  y_hat = [None for _ in range(train_samples)]
  for record in range(train_samples,train_samples+test_size):
    model = ARIMA(originalData, order=(p,d,q))
    model_fit = model.fit()
    prediction = model_fit.forecast()[0]
    y_hat.append(prediction)
    originalData.append(To_FilledValues['total_count'][record])
    originalData = originalData[1:]

  plt.figure(figsize=(15,5))
  plt.title("Predicted values vs Real values")
  plt.plot(To_FilledValues['total_count'][train_samples:train_samples+test_size], label="Real values")
  plt.plot(list(y_hat), color='red', label="Predicted values")
  plt.legend()
  plt.xlabel("Lags")
  plt.ylabel("Rentals")
  plt.grid(True)
  plt.savefig(f'prediction {city_data[0]}')
  plt.clf()

  # plot residual errors
  residuals = pd.DataFrame(model_fit.resid)
  residuals.plot()
  plt.title(f'Residuals - {city_data[0]}')
  plt.xlabel("Residual Error")
  plt.ylabel("Residuals")
  plt.grid(True)
  plt.savefig(f'Residuals {city_data[0]}')
  plt.clf()
  residuals.plot(kind='kde')
  plt.title(f'Density of Residuals - {city_data[0]}')
  plt.xlabel("Residual Error")
  plt.ylabel("Density")
  plt.grid(True)
  plt.savefig(f'Density of Residuals {city_data[0]}')
  plt.clf()
  return y_hat, model_fit


#### plotting the last 10 days that have actual records and predictions

In [687]:
fitted_models = []
for city_data in cities_df_array:
  y_hat, model_fit = Predictor(city_data)
  fitted_models.append((city_data[0],model_fit,y_hat))

/usr/local/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood op

<Figure size 1500x500 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 1500x500 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 1500x500 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

### Part 6 vairant pdq

In [719]:
from sklearn.metrics import mean_squared_error,r2_score, mean_absolute_percentage_error
TO_ACTUAL_DATA = To_FilledValues['total_count'][train_samples:train_samples+test_size]
SEA_ACTUAL_DATA = SEA_FilledValues['total_count'][train_samples:train_samples+test_size]
STU_ACTUAL_DATA = STU_FilledValues['total_count'][train_samples:train_samples+test_size]
cities_actual_data = [(CITY_ENUM.TO.value,TO_ACTUAL_DATA),(CITY_ENUM.SEA.value,SEA_ACTUAL_DATA),(CITY_ENUM.STU.value,STU_ACTUAL_DATA)]
for i in range(3):
  y_hat = fitted_models[i][2][504:]
  model_fit = fitted_models[i][1]
  mse = mean_squared_error(cities_actual_data[i][1], y_hat)
  rmse = np.sqrt(mse)
  r2 = r2_score(cities_actual_data[i][1], y_hat)
  mape = mean_absolute_percentage_error(cities_actual_data[i][1], y_hat)
  print(f'{fitted_models[i][0]} - MSE: {mse:.2f}, RMSE: {rmse:.2f}, R2: {r2:.2f}, MAPE: {mape:.2f}')

Torino - MSE: 345.95, RMSE: 18.60, R2: 0.76, MAPE: 0.28
Seattle - MSE: 1547.50, RMSE: 39.34, R2: 0.40, MAPE: 0.84
Stuttgart - MSE: 1058.35, RMSE: 32.53, R2: 0.50, MAPE: 1.16
